In [13]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from absl import app, flags
import numpy as np

from bucketed_sequence import BucketedSequence

UNK = -1.0
FLAGS = flags.FLAGS

batch_size=64
epochs=20
lstm_units=100
dense_breadth=32

dataset_size=10000
val_size=1000
seqlen_mean=50
seqlen_stddev=200
buckets=10

def pad(seqs, maxlen):
    # NOTE: prepends data
    padded = np.array(pad_sequences(seqs, maxlen=maxlen, value=UNK, 
                                    dtype=seqs[0].dtype))
    return np.vstack([np.expand_dims(x, axis=0) for x in padded])

def gen_dataset(set_size):
    sequence_lengths = np.random.normal(loc=seqlen_mean, 
        scale=seqlen_stddev, size=set_size).astype(np.int32)
    max_length = np.max(sequence_lengths)
    # Clamp range to start from three elements
    sequence_lengths = np.clip(sequence_lengths, 3, max_length)   
    
    # Generate random sequences
    seq_x  = [np.random.uniform(1.0, 50.0, sl) for sl in sequence_lengths]
    seq_y = np.array([seq[2] for seq in seq_x], dtype=np.float32)
    
    # Pad sequences
    padded_x = pad(seq_x, max_length)
    padded_x = np.reshape(padded_x,(len(sequence_lengths),max_length,1))
    
    # Return dataset
    return padded_x, seq_y, sequence_lengths

In [19]:
import sys
# Trains an LSTM to return the third (non-UNK) value in a sequence
def main():
    # Set up a simple network (LSTM + Dense)
    inp = Input(shape=(None, 1), dtype="float32", name="in")
    lstm = LSTM(lstm_units, return_sequences=False,
                name="lstm")(inp)
    dense = Dense(dense_breadth, kernel_initializer='normal',
                  activation='relu')(lstm)
    outputs = Dense(1, kernel_initializer='normal')(dense)
    model = Model(inputs=inp, outputs=outputs)
    model.compile(optimizer="adam", loss="mean_squared_error", 
                  metrics=['acc'])
    
    # Generate dataset
    x_train, y_train, len_train = gen_dataset(dataset_size)
    x_val, y_val, len_val = gen_dataset(val_size)
    
    
    if buckets > 0:
        # Create Sequence objects
        train_generator = BucketedSequence(buckets, batch_size,
                                           len_train, x_train, y_train)
        val_generator = BucketedSequence(buckets, batch_size,
                                         len_val, x_val, y_val)

        model.fit_generator(train_generator, epochs=epochs,
                            validation_data=val_generator,
                            shuffle=True, verbose=True)
    else:
        # No bucketing
        model.fit(x=x_train, y=y_train, epochs=epochs,
                  validation_data=(x_val, y_val),
                  batch_size=batch_size, verbose=True, shuffle=True)


In [18]:
main()

Training with 10 non-empty buckets
Training with 10 non-empty buckets
Epoch 1/20
162/162 [==============================] - 21s 132ms/step - loss: 400.0725 - mean_absolute_error: 16.2405 - val_loss: 186.1723 - val_mean_absolute_error: 11.7849
Epoch 2/20
162/162 [==============================] - 22s 134ms/step - loss: 158.2037 - mean_absolute_error: 10.4362 - val_loss: 146.6945 - val_mean_absolute_error: 9.6929
Epoch 3/20
162/162 [==============================] - 22s 134ms/step - loss: 133.9122 - mean_absolute_error: 8.8780 - val_loss: 128.3245 - val_mean_absolute_error: 8.3114
Epoch 4/20
162/162 [==============================] - 22s 135ms/step - loss: 123.7964 - mean_absolute_error: 8.1046 - val_loss: 145.5040 - val_mean_absolute_error: 9.9641
Epoch 5/20
162/162 [==============================] - 21s 132ms/step - loss: 122.0130 - mean_absolute_error: 7.8249 - val_loss: 124.1336 - val_mean_absolute_error: 7.8111
Epoch 6/20
162/162 [==============================] - 22s 135ms/step - l

KeyboardInterrupt: 